# Grab a selection of census tract data
*This notebook provides an example of how you can extract features from an ArcGIS online resource and save them into a shapefile on your local machine.* 

→ A good resource for learning more is here: https://developers.arcgis.com/python/guide/working-with-feature-layers-and-features/

In [ ]:
#Import the GIS object and authenticate
from arcgis import GIS
gis = GIS('pro')

In [ ]:
#Use the API's content' helper to search for feature layers with keyword Census
results = gis.content.search(query='census  tract',item_type='Feature Layer',outside_org=True)
results

In [ ]:
#Have a look at the 4th result
results[3]

In [ ]:
#Save it as an arcgis "item" object
tractsItem = results[3]
type(tractsItem)

→ More info on the ArcGIS `item` object: https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.gis.toc.html#item

In [ ]:
#Get the layer from the item
tractsLyr = tractsItem.layers[0]
type(tractsLyr)

→ More info on the ArcGIS `layer` object: https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.gis.toc.html#layer<br>
→ More info on the ArcGIS `FeatureLayer` object: https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html#featurelayer

---
### Subsetting and downloading content
Now that we have what we want as a FeatureLayer object, we can query that layer for just the features we we want to download. Before diving into our query, we'll need to familiarize ourselves with the data. 

Below are a set of steps mirrored after ESRI's documentation on querying feature layers:<br>
https://developers.arcgis.com/python/guide/working-with-feature-layers-and-features/#Querying-feature-layers

* First, we could just examine the ESRI REST endpoint for this dataset. To do that, we'd just reveal the URL associated with the data layer...

In [ ]:
print(tractsLyr.url)

* Or we could remain in our Python coding environment and reveal key properties using API functions:

In [ ]:
#List the fields associated with the raster
for f in tractsLyr.properties.fields:
    print(f['name'],end=', ')

* Now we'll construct a query function, much like if we were invoking the REST interface, but instead using the API functions, which streamline these things. 

In [ ]:
#Query the tracts feature layer for records falling within Durham Co (FIPS 37063)
query_result = tractsLyr.query("FIPS LIKE '37063%'")
type(query_result)

→ More info on the `Feature Set` object: https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html#featureset

In [ ]:
#Reveal how many features were extracted: 
len(query_result)

In [ ]:
#Save the feature set as shapefile
outFN = query_result.save('.','MyFeatures')
print("Output saved as {}".format(outFN))

### Or, analyzing the data here, as a dataframe

In [ ]:
#Convert the feature set data as a dataframe
df = query_result.sdf
df.head()

* Note the output has a column called "SHAPE". These values are ArcGIS API `geometry` objects. 

#### Analyzing geometry

In [ ]:
#Get the value in the first row of the "SHAPE" column
feat = df.loc[0,'SHAPE']
type(feat)

https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.geometry.html#arcgis.geometry.Geometry.get_area

In [ ]:
#Get the area, in square miles
feat.get_area(method='GEODESIC',units='MILES')

#### Analyzing age demographics

In [ ]:
#Grab the first 10 columns into a new dataframe
ageColsDF = df.iloc[:,:9]

In [ ]:
#Summarize those columns
ageColsDF.describe()

In [ ]:
#Plot demographics: count within each age group
ageCols.sum()

In [ ]:
%matplotlib inline
ageCols.sum().plot(kind='bar');